## 🔎 Google AI Search: Simulating Query Fan-Out Visibility

This notebook explores how well a given URL's content might be visible to Google's AI search mechanisms, specifically by simulating its "query fan-out" process. Read more from our blog: "[Query Fan-Out AI Search](https://wordlift.io/blog/en/query-fan-out-ai-search/)"

**Core Idea (based on recent studies and insights from patents like US 2024/0289407 A1):**
1.  **Entity Identification:** Determine the main entity/topic of a URL.
2.  **Synthetic Query Generation (Fan-Out Simulation):** For this entity, generate a set of specific sub-queries that Google's AI might internally use to build a comprehensive understanding. This simulation is further informed by mechanisms detailed in patents such as **WO2024064249A1** (_Systems and methods for prompt-based query generation for diverse retrieval_). The relevance of such patent work to Google's AI Mode query fan-out processes has been highlighted by industry analysis, notably by [Michael King of iPullRank](https://ipullrank.com/how-ai-mode-works).
3.  **Content Coverage Assessment:** Check if the original URL's content effectively answers these synthetic sub-queries.

This approach, using `DSPy`, aims to move towards more robust AI system building, focusing on task-oriented modules rather than intricate prompt engineering.

> **Requirements** – A Google Gemini API key.

> **PS:** It’s early-stage. One thing missing is local context extraction (e.g., if the user has a Knowledge Graph with WordLift, we could check if expected passages are present on the website aside from the audited URL. **[Contact us](https://wordlift.io/book-a-demo/)** to learn more).

</br>
<table align="left">
  <td>
  <a href="https://wordlift.io">
    <img width=130px src="https://upload.wikimedia.org/wikipedia/commons/4/48/WordLift-logo-horizontal-2024.png" />
    </a>
    </td>
    <td>
      <i>Concept by WordLift</i>
      <br/>
      <i>Author: Andrea Volpini & AI Assistant</i>
      <br/>
      <i>Last updated: June 9th, 2025</i>
  </td>
</table>
</br>
</br>

### 1. Install Dependencies

In [1]:
print("--- Installing core libraries ---")
!pip install -U dspy-ai google-generativeai numpy -qq
print("--- Installation Complete ---")

# Verify installed versions of key packages
print("\n--- Key Installed Versions ---")
!pip list | grep dspy-ai
!pip list | grep google-generativeai
print("----------------------------")

--- Installing core libraries ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >

### 2. Setup API Keys & DSPy Configuration

In [9]:
# Cell 2: Setup API Keys & DSPy Configuration (Revised for Gemini + OpenAI Embeddings)

from google.colab import userdata # For Colab secrets
import os
import openai
from google import genai
import dspy as dspy # For DSPy
import requests # Will be used in utility functions
from urllib.parse import urlparse # Will be used in utility functions
import re # Will be used in utility functions
import numpy as np # Will be used in utility functions
import json # Will be used in utility functions
import datetime # Will be used in utility functions
from IPython.display import display, HTML # Will be used later
from google.colab import userdata
import datetime # For current_date

# ------------  API KEY SETUP  --------------
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') or userdata.get('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    print("❌ ERROR: GEMINI_API_KEY not found. All Gemini operations will fail.")
else:
    # Set for LiteLLM as a fallback, but we will pass the key directly.
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    print("✅ GEMINI_API_KEY configured.")

# ------------  SHARED GEMINI CLIENT & MODEL DEFINITIONS FOR UTILITIES  --------------
gemini_sdk_client_for_utils = None
MODEL_FOR_URL_CONTEXT_TOOL = "gemini-2.5-flash-preview-05-20"
GEMINI_EMBEDDING_MODEL_FOR_CLIENT = "models/text-embedding-004"

if GEMINI_API_KEY:
    try:
        gemini_sdk_client_for_utils = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Shared `genai.Client` initialized for utilities.")
    except Exception as e:
        print(f"❌ ERROR: Failed to initialize `genai.Client`: {e}")

# ------------  DSPY CONFIGURATION (for Query Fan-Out)  --------------
GEMINI_DSPY_LLM_MODEL = "gemini/gemini-2.5-flash-preview-05-20"
CONCEPTUAL_GEMINI_EMBEDDING_MODEL_FOR_DSPY = f"gemini/{GEMINI_EMBEDDING_MODEL_FOR_CLIENT.replace('models/','')}"

if GEMINI_API_KEY:
    try:
        # ** THE KEY FIX IS HERE: Explicitly pass api_key to dspy.LM **
        llm_for_dspy = dspy.LM(
            model=GEMINI_DSPY_LLM_MODEL,
            api_key=GEMINI_API_KEY,  # Pass the key directly
            max_tokens=1500,
            temperature=0.5
        )
        # Also pass the key to the conceptual RM for completeness
        rm_for_dspy = dspy.LM(
            model=CONCEPTUAL_GEMINI_EMBEDDING_MODEL_FOR_DSPY,
            api_key=GEMINI_API_KEY # Pass the key directly
        )

        dspy.settings.configure(lm=llm_for_dspy, rm=rm_for_dspy)
        print("✅ DSPy configured successfully.")
    except Exception as e:
        print(f"❌ ERROR: Failed to configure DSPy with Gemini models: {e}")
        if 'dspy' in locals() and hasattr(dspy, 'settings'): dspy.settings.configure(lm=None, rm=None)
else:
    print("⚠️ DSPy not configured because GEMINI_API_KEY is missing.")


# ------------ FINAL CONFIGURATION CHECK & REPORTING VARIABLES ------------
# These variables are set here to be used in the final report in Cell 5
if 'dspy' in locals() and hasattr(dspy.settings, 'lm') and dspy.settings.lm and hasattr(dspy.settings.lm, 'model'):
    JSON_REPORT_LLM_PROVIDER = "Gemini (DSPy LM)"
    JSON_REPORT_LLM_MODEL = dspy.settings.lm.model
    print(f"  - LLM for DSPy Fan-Out: ✅ {JSON_REPORT_LLM_PROVIDER} -> {JSON_REPORT_LLM_MODEL}")
else:
    JSON_REPORT_LLM_PROVIDER = "N/A"
    JSON_REPORT_LLM_MODEL = "N/A"
    print("  - LLM for DSPy Fan-Out: ❌ Not configured.")

if 'gemini_sdk_client_for_utils' in globals() and gemini_sdk_client_for_utils:
    JSON_REPORT_EMBEDDING_PROVIDER = "Gemini (Direct Client)"
    JSON_REPORT_EMBEDDING_MODEL = GEMINI_EMBEDDING_MODEL_FOR_CLIENT
    print(f"  - Client for Utilities: ✅ Initialized")
    print(f"     - URL Insight Model: {MODEL_FOR_URL_CONTEXT_TOOL}")
    print(f"     - Embedding Model: {JSON_REPORT_EMBEDDING_MODEL}")
else:
    JSON_REPORT_EMBEDDING_PROVIDER = "N/A"
    JSON_REPORT_EMBEDDING_MODEL = "N/A"
    print("  - Client for Utilities: ❌ FAILED to initialize.")

✅ GEMINI_API_KEY configured.
✅ Shared `genai.Client` initialized for utilities.
✅ DSPy configured successfully.
  - LLM for DSPy Fan-Out: ✅ Gemini (DSPy LM) -> gemini/gemini-2.5-flash-preview-05-20
  - Client for Utilities: ✅ Initialized
     - URL Insight Model: gemini-2.5-flash-preview-05-20
     - Embedding Model: models/text-embedding-004


### 3. Utility Functions

In [10]:
# Cell 3: Utility Functions (Gemini-Only)

def get_url_insights_and_content_via_gemini_client(url_to_process: str) -> dict:
    """
    Uses the shared genai.Client with its url_context tool to get the main entity
    and grounded content chunks from a URL.
    This function is based on your previously confirmed working logic.
    """
    global gemini_sdk_client_for_utils, MODEL_FOR_URL_CONTEXT_TOOL

    if not gemini_sdk_client_for_utils:
        error_msg = "Gemini SDK Client not initialized. Run Cell 2 first."
        print(f"❌ CRITICAL ERROR: {error_msg}")
        return {"status": "failure", "entity": None, "content_chunks": [], "error_message": error_msg}

    print(f"Processing URL with genai.Client ({MODEL_FOR_URL_CONTEXT_TOOL}): {url_to_process}")

    prompt = f"Analyze the content of the webpage at: {url_to_process}. Then, identify and state the primary subject or main entity of this page. Respond concisely with only the entity name."

    try:
        response = gemini_sdk_client_for_utils.models.generate_content(
            model=MODEL_FOR_URL_CONTEXT_TOOL,
            contents=[prompt],
            config={"tools": [{"url_context": {}}]}
        )

        entity_text = None
        retrieved_chunks = []

        if response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
            entity_text = response.candidates[0].content.parts[0].text.strip()

        if hasattr(response.candidates[0], 'grounding_metadata') and response.candidates[0].grounding_metadata.grounding_chunks:
            chunks_data = response.candidates[0].grounding_metadata.grounding_chunks
            print(f"✅ Found {len(chunks_data)} grounded content chunks.")
            for chunk in chunks_data:
                if hasattr(chunk, 'retrieved_context') and chunk.retrieved_context and hasattr(chunk.retrieved_context, 'text'):
                    retrieved_chunks.append(chunk.retrieved_context.text.strip())
                elif hasattr(chunk, 'web') and chunk.web and hasattr(chunk.web, 'title'):
                    retrieved_chunks.append(chunk.web.title.strip())

        if not retrieved_chunks:
            if entity_text: # If we got a text response, use it as a chunk.
                print("⚠️ No specific grounded chunks found. Using model's text response as content.")
                retrieved_chunks = [entity_text]
            else: # If we got no text and no chunks, it's a failure.
                error_msg = "Gemini did not return any valid content or grounded chunks from the URL."
                print(f"❌ {error_msg}")
                return {"status": "failure", "entity": None, "content_chunks": [], "error_message": error_msg}

        if not entity_text:
            print("  -> No direct entity text found. Inferring from chunks...")
            entity_prompt = f"Based on this text, what is the main entity? Respond with only the entity name.\n\nTEXT: {retrieved_chunks[0][:1000]}"
            entity_response = genai.GenerativeModel("gemini-2.5-flash").generate_content(entity_prompt)
            if entity_response.text: entity_text = entity_response.text.strip()

        if entity_text:
             entity_text = re.sub(r"^(The main entity.*?is\s*:?\s*)+", "", entity_text, flags=re.IGNORECASE).strip().split('\n')[0].strip('."\' ')

        parsed_domain = urlparse(url_to_process).netloc.replace("www.","").split('.')[0].title()
        if not entity_text or (parsed_domain.lower() in entity_text.lower() and len(entity_text.split()) < 4):
            entity_text = parsed_domain

        print(f"✅ Gemini identified entity: '{entity_text}'")
        return {"status": "success", "entity": entity_text, "content_chunks": retrieved_chunks}

    except Exception as e:
        error_msg = f"Error during Gemini API call for URL insights: {e}"
        print(f"❌ {error_msg}")
        return {"status": "failure", "entity": None, "content_chunks": [], "error_message": error_msg}


def get_embedding(text: str) -> np.ndarray:
    """
    Generate embedding for text using the shared genai.Client and Gemini embedding model.
    This version uses the correct `contents` parameter and has the unsupported 'task_type' removed.
    """
    global gemini_sdk_client_for_utils, GEMINI_EMBEDDING_MODEL_FOR_CLIENT

    if not text or not gemini_sdk_client_for_utils:
        if not gemini_sdk_client_for_utils:
            print("❌ CRITICAL ERROR in get_embedding: Shared genai.Client not initialized.")
        return np.array([])

    try:
        # ** THE FINAL, CORRECTED API CALL **
        # No 'task_type' argument.
        result = gemini_sdk_client_for_utils.models.embed_content(
            model=GEMINI_EMBEDDING_MODEL_FOR_CLIENT,
            contents=[text.strip()] # Correctly using 'contents' with a list
        )

        # The response for a list of contents is an object with an '.embeddings' attribute.
        if hasattr(result, 'embeddings') and result.embeddings:
            # result.embeddings is a list of embedding objects. We want the values from the first one.
            embedding_object = result.embeddings[0]
            if hasattr(embedding_object, 'values'):
                 return np.array(embedding_object.values).astype('float32')

        # Fallback if the structure is not as expected
        print(f"⚠️ Warning: Gemini embedding response format unexpected or empty for '{text[:50]}...'.")
        return np.array([])

    except Exception as e:
        print(f"❌ Error in get_embedding with Gemini for '{text[:50]}...': {e}")
        return np.array([])


def cosine_similarity(vec1, vec2):
    if not isinstance(vec1, np.ndarray) or not isinstance(vec2, np.ndarray) or vec1.size == 0 or vec2.size == 0: return 0.0
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0: return 0.0
    return dot_product / (norm_vec1 * norm_vec2)


def chunk_text(text, max_chunk_length=500, overlap_words=10):
    if not text: return []
    sentences = re.split(r'(?<=[.!?])\\s+', text.replace('\n', ' '))
    if not sentences: return [text] if text else []
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) > max_chunk_length and current_chunk:
            chunks.append(current_chunk.strip())
            current_chunk = " ".join(current_chunk.split()[-overlap_words:])
        current_chunk += " " + sentence
    if current_chunk: chunks.append(current_chunk.strip())
    return [c for c in chunks if c]

### 4. DSPy Signatures and Modules for Fan-Out Simulation (with ChainOfThought)

In [11]:
# Cell 4: DSPy Signatures and Modules for Fan-Out Simulation

class GenerateSyntheticQueriesCoT(dspy.Signature):
    """
    Given an entity (typically the main topic of a webpage), the current date, and a desired number of sub-queries,
    first, briefly reason about the key facets, common questions, and closely related concepts
    a user exploring this entity with an AI search companion (like Google AI Mode) might be interested in.
    Then, generate a list of diverse, specific, and factual sub-queries that reflect this exploration.

    These sub-queries simulate how an AI might "fan-out" its internal searches.
    Consider the current date: {current_date} to ensure queries aim for up-to-date information if relevant to the entity.

    For the reasoning, identify 3-5 primary facets or types of information related to the entity.
    For the synthetic queries, aim for a mix that explores these facets. For example, consider:
    - Definitional/Explanatory queries about the entity or its core components.
    - Practical application or "how-to" queries related to the entity.
    - Queries exploring benefits, drawbacks, or comparisons if applicable.
    - Queries about recent developments or current status, especially if the entity is time-sensitive, informed by the current date.
    - Queries touching on closely related sub-topics or entities.

    The final output for 'synthetic_queries' MUST be a Python-parseable list of strings.
    Example for entity "Agile Project Management", num_queries "4", and current_date "{current_date}":
    Reasoning About Facets: Key facets for Agile Project Management include its core principles, benefits particularly in software development, common tools used by teams, and recent trends or discussions around its adoption as of {current_date}.
    Synthetic Queries: ["what are the core principles of agile project management", "how does agile project management benefit software development teams", "popular agile project management tools in {current_year_from_date}", "latest trends in agile methodology adoption as of {current_date}"]
    """
    entity_name = dspy.InputField(desc="The main entity or topic of a webpage.")
    current_date = dspy.InputField(desc="The current date, e.g., 'May 27, 2025'.")
    num_queries_to_generate = dspy.InputField(desc="The exact number of distinct sub-queries to generate (e.g., '5').")

    reasoning_about_facets = dspy.OutputField(desc="Briefly outline the key information facets or types of user questions considered for the entity. Explain their relevance for a comprehensive understanding, considering the current date if applicable.")
    synthetic_queries = dspy.OutputField(desc="A Python list of strings, containing exactly the number of sub-queries specified. Format: [\"query1\", \"query2\", ...]")

class GoogleAIQueryFanOut(dspy.Module):
    def __init__(self, num_queries_default=5):
        super().__init__()
        self.num_queries_default = num_queries_default
        self.generate_queries_with_cot = dspy.ChainOfThought(GenerateSyntheticQueriesCoT)

    def forward(self, entity_name, current_date_str, num_queries=None):
        num_to_gen_val = num_queries if num_queries is not None else self.num_queries_default

        prediction = self.generate_queries_with_cot(
            entity_name=entity_name,
            current_date=current_date_str,
            num_queries_to_generate=str(num_to_gen_val)
        )

        reasoning_output = "No reasoning provided."
        if hasattr(prediction, 'reasoning_about_facets'):
            reasoning_output = prediction.reasoning_about_facets
            print(f"🧠 Reasoning for '{entity_name}' (Date: {current_date_str}, Num queries: {num_to_gen_val}):\n{reasoning_output}")

        raw_output = prediction.synthetic_queries.strip()
        queries = []

        if raw_output.startswith('[') and raw_output.endswith(']'):
            try:
                parsed_list = json.loads(raw_output)
                if isinstance(parsed_list, list) and all(isinstance(item, str) for item in parsed_list):
                    queries = parsed_list
                    # print(f"Successfully parsed queries using json.loads for '{entity_name}'.")
            except json.JSONDecodeError:
                # print(f"json.loads failed for raw_output: '{raw_output}'. Trying other parsing methods.")
                pass

        if not queries:
            list_match = re.search(r"\[\s*('([^']*)'|\"([^\"]*)\")\s*(,\s*('([^']*)'|\"([^\"]*)\"))*\s*\]", raw_output)
            if list_match:
                list_str_content = list_match.group(0)
                try:
                    import ast
                    evaluated_list = ast.literal_eval(list_str_content)
                    if isinstance(evaluated_list, list) and all(isinstance(item, str) for item in evaluated_list):
                        queries = evaluated_list
                        # print(f"Successfully parsed queries using ast.literal_eval for '{entity_name}'.")
                except (ValueError, SyntaxError, TypeError):
                    # print(f"ast.literal_eval failed for list_str_content: '{list_str_content}'. Trying newline split.")
                    pass

            if not queries:
                potential_queries = raw_output.splitlines()
                cleaned_queries = []
                for q_line in potential_queries:
                    q_line = q_line.strip()
                    q_line = re.sub(r"^\s*(\d+\.|-|\*)\s*", "", q_line)
                    if (q_line.startswith('"') and q_line.endswith('"')) or \
                       (q_line.startswith("'") and q_line.endswith("'")):
                        q_line = q_line[1:-1]
                    if q_line:
                        cleaned_queries.append(q_line)
                if cleaned_queries:
                    queries = cleaned_queries
                    # print(f"Parsed queries using newline splitting for '{entity_name}'.")

        queries = [str(q).strip() for q in queries if str(q).strip()][:num_to_gen_val]
        if not queries and raw_output:
            queries = [raw_output]
            print(f"Warning: All parsing failed for '{entity_name}'. Treating raw output as a single query: '{raw_output[:100]}...'")

        return dspy.Prediction(
            synthetic_queries=queries,
            reasoning_about_facets=reasoning_output
        )

class ContentCoverageScorer(dspy.Module):
    def __init__(self, similarity_threshold_default=0.75):
        super().__init__()
        self.similarity_threshold_default = similarity_threshold_default

    def forward(self, query, content_chunks, similarity_threshold=None):
        threshold = similarity_threshold if similarity_threshold is not None else self.similarity_threshold_default
        if not query or not content_chunks:
            return dspy.Prediction(is_covered=False, best_matching_chunk="", max_similarity=0.0)

        query_embedding = get_embedding(str(query))
        if query_embedding.size == 0:
            return dspy.Prediction(is_covered=False, best_matching_chunk="", max_similarity=0.0)

        max_similarity = 0.0
        best_chunk_for_query = ""
        covered_by_threshold = False

        for chunk_text_item in content_chunks:
            chunk_embedding = get_embedding(str(chunk_text_item))
            if chunk_embedding.size > 0:
                sim = cosine_similarity(query_embedding, chunk_embedding)
                if sim > max_similarity:
                    max_similarity = sim
                    best_chunk_for_query = str(chunk_text_item)
                if sim >= threshold:
                    covered_by_threshold = True

        return dspy.Prediction(is_covered=covered_by_threshold, best_matching_chunk=best_chunk_for_query if covered_by_threshold else "", max_similarity=float(max_similarity))

class AIVisibilityAuditor(dspy.Module):
    def __init__(self, num_synthetic_queries_default=5, coverage_threshold_default=0.75):
        super().__init__()
        self.query_fanout = GoogleAIQueryFanOut(num_queries_default=num_synthetic_queries_default)
        self.coverage_scorer = ContentCoverageScorer(similarity_threshold_default=coverage_threshold_default)

    def forward(self, url, num_synthetic_queries=None, coverage_threshold=None):
        print(f"\n🚀 Starting AI Visibility Audit for: {url}")

        url_insights = get_url_insights_and_content_via_gemini_client(url)

        if url_insights.get("status") == "failure":
            error_msg = url_insights.get("error_message", "Unknown Gemini insight error")
            print(f"❌ Failed to get insights from Gemini for {url}: {error_msg}. Aborting audit.")
            return dspy.Prediction(
                entity_name="Gemini Insight Error",
                url=url,
                coverage_score=0.0,
                audit_details=[],
                reasoning_about_facets=f"Gemini Insight Error: {error_msg}"
            )

        main_entity = url_insights.get("entity")
        initial_content_chunks = url_insights.get("content_chunks", [])

        if not main_entity:
            print(f"❌ Critical: No main entity could be determined by Gemini for {url}. Cannot generate synthetic queries.")
            return dspy.Prediction(entity_name="Entity Extraction Failed (Gemini)", url=url, coverage_score=0.0, audit_details=[], reasoning_about_facets="Entity Extraction Failed (Gemini)")

        print(f"🧠 Main entity identified by Gemini: '{main_entity}'")

        TOKEN_LIMIT_PROXY_CHARS = 6000
        final_content_chunks = []

        if initial_content_chunks:
            print(f"📄 Obtained {len(initial_content_chunks)} grounded content chunk(s) from Gemini. Validating for embedding limits...")
            for chunk_item in initial_content_chunks:
                if len(str(chunk_item)) > TOKEN_LIMIT_PROXY_CHARS:
                    print(f"  -> Chunk is too long ({len(str(chunk_item))} chars). Re-chunking it...")
                    sub_chunks = chunk_text(str(chunk_item), max_chunk_length=TOKEN_LIMIT_PROXY_CHARS, overlap_words=50)
                    final_content_chunks.extend(sub_chunks)
                elif str(chunk_item).strip():
                    final_content_chunks.append(str(chunk_item))
            print(f"  -> Processed into {len(final_content_chunks)} embedding-safe chunks.")
        else:
            print("⚠️ Gemini provided an entity but no content chunks. Coverage will be 0 if no other chunks are found.")

        content_chunks_for_scoring = final_content_chunks

        num_queries_to_run = num_synthetic_queries if num_synthetic_queries is not None else self.query_fanout.num_queries_default
        current_date_str = datetime.datetime.now().strftime("%B %d, %Y")

        synthetic_queries_pred = self.query_fanout( # CORRECTED CALL
            entity_name=str(main_entity),
            current_date_str=current_date_str,      # Pass current_date_str
            num_queries=num_queries_to_run
        )
        synthetic_queries = synthetic_queries_pred.synthetic_queries
        reasoning_text = synthetic_queries_pred.reasoning_about_facets if hasattr(synthetic_queries_pred, 'reasoning_about_facets') else "N/A"

        print(f"🔍 Generated {len(synthetic_queries)} synthetic queries for '{main_entity}':")
        for i, q_text in enumerate(synthetic_queries):
            print(f"  {i+1}. {q_text}")

        if not synthetic_queries or not content_chunks_for_scoring:
            final_coverage_score = 0.0
            audit_details = [{'query': sq, 'covered': False, 'max_similarity': 0.0, 'best_chunk': ''} for sq in synthetic_queries] if synthetic_queries else []
            if not synthetic_queries:
                print("No synthetic queries to assess.")
            if not content_chunks_for_scoring and synthetic_queries:
                print("No content chunks to assess against generated queries. Coverage will be 0.")
        else:
            covered_count = 0
            audit_details = []
            print("\n🕵️ Assessing coverage for each synthetic query...")

            for query_text_item in synthetic_queries:
                current_coverage_threshold = coverage_threshold if coverage_threshold is not None else self.coverage_scorer.similarity_threshold_default

                coverage_pred = self.coverage_scorer(
                    query=query_text_item,
                    content_chunks=content_chunks_for_scoring,
                    similarity_threshold=current_coverage_threshold
                )
                status = "✅ Covered" if coverage_pred.is_covered else "❌ Not Covered"
                print(f"  - Query: '{str(query_text_item)[:70]}...' -> {status} (Max Similarity: {coverage_pred.max_similarity:.2f})")

                if coverage_pred.is_covered:
                    covered_count += 1
                audit_details.append({
                    'query': query_text_item,
                    'covered': coverage_pred.is_covered,
                    'max_similarity': coverage_pred.max_similarity,
                    'best_chunk': coverage_pred.best_matching_chunk
                })
            final_coverage_score = (covered_count / len(synthetic_queries)) * 100 if synthetic_queries else 0.0

        print(f"\n📊 Final Coverage Score for '{main_entity}' on {url}: {final_coverage_score:.2f}%")

        return dspy.Prediction(
            entity_name=str(main_entity),
            url=url,
            coverage_score=final_coverage_score,
            audit_details=audit_details,
            reasoning_about_facets=reasoning_text
        )

### 5. Run the Audit with Interactive Inputs (updated with CoT reasoning)

In [12]:
# Cell 5: Run the Audit with Interactive Inputs

#@title Run AI Visibility Audit
#@markdown Enter the URL to analyze and adjust the parameters for the simulation.
url_to_audit_form = 'https://francescoragusa.com/content-audit/' #@param {type:"string"}
num_synthetic_queries_form = 20 #@param {type:"integer", min:3, max:10, step:1}
coverage_threshold_form = 0.65 #@param {type:"number", min:0.5, max:0.9, step:0.01}

# --- Pre-run Check ---
# Check if the necessary clients and settings are configured from Cell 2
can_run_audit = True
if not 'GEMINI_API_KEY' in globals() or not GEMINI_API_KEY:
    print("❌ ERROR: GEMINI_API_KEY is not configured. Please set it in Cell 2 and re-run.")
    can_run_audit = False
if not 'dspy' in globals() or not hasattr(dspy.settings, 'lm') or not dspy.settings.lm:
    print("❌ ERROR: DSPy Language Model (lm) is not configured. Please run Cell 2.")
    can_run_audit = False
if not 'gemini_sdk_client_for_utils' in globals() or not gemini_sdk_client_for_utils:
    print("❌ ERROR: Shared Gemini Client (for utilities) is not initialized. Please run Cell 2.")
    can_run_audit = False

if can_run_audit:
    # --- Execute Audit ---
    # Instantiate the auditor with default values from the form.
    # The forward method will use these unless overridden.
    auditor = AIVisibilityAuditor(
        num_synthetic_queries_default=num_synthetic_queries_form,
        coverage_threshold_default=coverage_threshold_form
    )

    audit_result = auditor(url=url_to_audit_form)

    # --- Process and Display Results ---
    if audit_result:
        # Prepare JSON output for a detailed report
        output_data = {
            'url': audit_result.url,
            'entity_name': audit_result.entity_name,
            'overall_coverage_score_percent': audit_result.coverage_score,
            'audit_timestamp_utc': datetime.datetime.utcnow().isoformat(),
            'parameters': {
                'num_synthetic_queries_requested': num_synthetic_queries_form,
                'num_synthetic_queries_generated': len(audit_result.audit_details) if audit_result.audit_details else 0,
                'coverage_threshold_used': coverage_threshold_form
            },
            'llm_reasoning_for_queries': audit_result.reasoning_steps if hasattr(audit_result, 'reasoning_steps') else "N/A",
            'synthetic_query_details': audit_result.audit_details,
            'llm_provider': JSON_REPORT_LLM_PROVIDER, # Variable from Cell 2
            'llm_model_used': JSON_REPORT_LLM_MODEL, # Variable from Cell 2
            'embedding_provider': JSON_REPORT_EMBEDDING_PROVIDER, # Variable from Cell 2
            'embedding_model_used': JSON_REPORT_EMBEDDING_MODEL # Variable from Cell 2
        }

        # Create a safe filename for the JSON report
        entity_file_name_part = "audit_report" # Default
        if audit_result.entity_name and isinstance(audit_result.entity_name, str) and "Error" not in audit_result.entity_name:
             entity_file_name_part = re.sub(r'[^a-zA-Z0-9_]', '', audit_result.entity_name.replace(' ', '_')).lower()[:30]
        output_filename = f"ai_visibility_audit_{entity_file_name_part}.json"

        with open(output_filename, 'w') as f:
            json.dump(output_data, f, indent=2)
        print(f"\n💾 Audit results saved to {output_filename}")

        # Display a user-friendly HTML summary in the notebook
        html_summary = f"<h1>AI Visibility Audit: {audit_result.entity_name}</h1>"
        html_summary += f"<p><b>URL:</b> <a href='{audit_result.url}' target='_blank'>{audit_result.url}</a></p>"
        html_summary += f"<p><b>Overall AI Query Fan-Out Coverage:</b> {audit_result.coverage_score:.2f}%</p>"

        if hasattr(audit_result, 'reasoning_steps') and audit_result.reasoning_steps and audit_result.reasoning_steps != "N/A":
            escaped_reasoning = audit_result.reasoning_steps.replace('&', '&').replace('<', '<').replace('>', '>')
            html_summary += f"<h3>LLM Reasoning for Query Generation:</h3><pre style='white-space: pre-wrap; background-color: #f0f0f0; padding: 10px; border-radius: 5px; font-family: monospace;'>{escaped_reasoning}</pre>"

        html_summary += "<h3>Synthetic Query Coverage Details:</h3><ul>"
        if audit_result.audit_details:
            for detail in audit_result.audit_details:
                status_emoji = "✅" if detail['covered'] else "❌"
                escaped_query = str(detail['query']).replace('&', '&').replace('<', '<').replace('>', '>')
                html_summary += f"<li>{status_emoji} <b>Query:</b> {escaped_query} (Max Similarity: {detail['max_similarity']:.2f})</li>"
        else:
            html_summary += "<li>No synthetic queries were processed or available.</li>"
        html_summary += "</ul>"

        display(HTML(html_summary))

        # Offer the JSON report for download in Colab
        try:
            from google.colab import files
            files.download(output_filename)
        except ImportError:
            print(f"To download the report outside Colab, find '{output_filename}' in your current directory.")
    else:
        print("Audit did not produce a result. Please check the log for errors in previous steps.")


🚀 Starting AI Visibility Audit for: https://francescoragusa.com/content-audit/
Processing URL with genai.Client (gemini-2.5-flash-preview-05-20): https://francescoragusa.com/content-audit/
✅ Found 1 grounded content chunks.
✅ Gemini identified entity: 'The webpage is titled "Content Audit: guida + Template Gratuito - Francesco Ragusa" and discusses in detail what a content audit is, its purpose in SEO, how to perform one, and how to make decisions about website content. It also mentions content pruning and provides a free template. The entire content revolves around the topic of content auditing'
🧠 Main entity identified by Gemini: 'The webpage is titled "Content Audit: guida + Template Gratuito - Francesco Ragusa" and discusses in detail what a content audit is, its purpose in SEO, how to perform one, and how to make decisions about website content. It also mentions content pruning and provides a free template. The entire content revolves around the topic of content auditing'
📄 Obtai

/tmp/ipython-input-852474179.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'audit_timestamp_utc': datetime.datetime.utcnow().isoformat(),


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>